#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [47]:
import pandas as pd
import pymongo

In [55]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [146]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [57]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [75]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

In [147]:
df_ks['goal'] = pd.to_numeric(df_ks['goal'], errors='coerce')
df_ks['pledged'] = pd.to_numeric(df_ks['pledged'], errors='coerce')
df_ks['backers'] = pd.to_numeric(df_ks['backers'], errors='coerce')
df_ks['usd pledged'] = pd.to_numeric(df_ks['usd pledged'], errors='coerce')

In [133]:
# df_ks.dropna(inplace = True, subset = ['goal', 'pledged', 'backers', 'usd pledged'])

In [134]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers             float64
country              object
usd pledged         float64
usd_pledged_real    float64
dtype: object

In [148]:
data_dict = df_ks.to_dict("records")

In [94]:
data_dict

[{'ID': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2011-09-16',
  'goal': 2000.0,
  'launched': '2011-08-17 06:31:31',
  'pledged': 1145.0,
  'state': 'canceled',
  'backers': 24.0,
  'country': 'US',
  'usd pledged': 1145.0,
  'usd_pledged_real': 1145.0},
 {'ID': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  'launched': '2012-07-23 20:46:48',
  'pledged': 1851.0,
  'state': 'failed',
  'backers': 28.0,
  'country': 'US',
  'usd pledged': 1851.0,
  'usd_pledged_real': 1851.0},
 {'ID': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2016-07-19',
  'goal': 2000.0,
  'launched': '2016-07-01 21:55:54',
  'pledged': 7534.0,
  '

### Importer les données

In [149]:
#Si besoin : 
collection.delete_many({})

In [150]:
collection.insert_many(data_dict)

In [96]:
print(client.list_database_names())

['admin', 'config', 'exercices', 'local', 'series']


## Question 1  
Récupérer les 5 projets ayant reçu le plus de promesse de dons.

In [155]:
cur = collection.find().sort([("pledged", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('61dd8dd53bd217711a731f06'),
  'ID': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': 50000.0,
  'launched': '2014-07-08 10:14:37',
  'pledged': 13285226.36,
  'state': 'successful',
  'backers': 62642.0,
  'country': 'US',
  'usd pledged': 13285226.36,
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('61dd8dd53bd217711a7379ff'),
  'ID': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': 1000000.0,
  'launched': '2016-05-24 15:49:52',
  'pledged': 12779843.49,
  'state': 'successful',
  'backers': 66673.0,
  'country': 'US',
  'usd pledged': 12779843.49,
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('61dd8dd53bd217711a732b3d'),
  'ID': 2111201788,
  'name': 'Expect the Unex

## Question 2
Compter le nombre de projets ayant atteint leur but.


In [159]:
count = collection.count({'state':'successful'})
ratio = count/collection.count()
print("count :",count,"(",ratio*100,"%)")

count : 53040 ( 35.36 %)


/tmp/ipykernel_15/3333847334.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  count = collection.count({'state':'successful'})
/tmp/ipykernel_15/3333847334.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  ratio = count/collection.count()


## Question 3
Compter le nombre de projets pour chaque catégorie.

In [178]:
cat_count = collection.aggregate([{"$group" : {"_id" : "$category", "count":{"$sum":1}}},{"$sort": {"count":-1}} ])
list(cat_count)

[{'_id': 'Product Design', 'count': 8886},
 {'_id': 'Documentary', 'count': 6498},
 {'_id': 'Music', 'count': 6229},
 {'_id': 'Tabletop Games', 'count': 5581},
 {'_id': 'Shorts', 'count': 4857},
 {'_id': 'Video Games', 'count': 4799},
 {'_id': 'Food', 'count': 4612},
 {'_id': 'Film & Video', 'count': 4003},
 {'_id': 'Fiction', 'count': 3703},
 {'_id': 'Nonfiction', 'count': 3390},
 {'_id': 'Fashion', 'count': 3379},
 {'_id': 'Art', 'count': 3358},
 {'_id': 'Apparel', 'count': 2827},
 {'_id': 'Theater', 'count': 2786},
 {'_id': 'Rock', 'count': 2707},
 {'_id': 'Technology', 'count': 2690},
 {'_id': "Children's Books", 'count': 2686},
 {'_id': 'Apps', 'count': 2535},
 {'_id': 'Publishing', 'count': 2332},
 {'_id': 'Webseries', 'count': 2316},
 {'_id': 'Photography', 'count': 2239},
 {'_id': 'Indie Rock', 'count': 2192},
 {'_id': 'Narrative Film', 'count': 2099},
 {'_id': 'Web', 'count': 2017},
 {'_id': 'Comics', 'count': 1931},
 {'_id': 'Crafts', 'count': 1834},
 {'_id': 'Country & Folk'

## Question 4
Compter le nombre de projets français ayant été instanciés avant 2016.

In [189]:
collection.count({"$and":[{"country":"FR"},{ "launched" : { "$lt":"2016-00-00 00:00:00"}}]})


/tmp/ipykernel_15/2447115280.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  collection.count({"$and":[{"country":"FR"},{ "launched" : { "$lt":"2016-00-00 00:00:00"}}]})


330

## Question 5
Récupérer les projets américains ayant demandé plus de 200 000 dollars.

In [194]:
l = collection.find({"$and":[{"country":"US"},{"goal":{"$gt":200000}}]})
print("Nombre : ",l.count())
list(l)

/tmp/ipykernel_15/704656865.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print("Nombre : ",l.count())


Nombre :  1841


[{'_id': ObjectId('61dd8dd43bd217711a71f483'),
  'ID': 866634482,
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2012-09-14',
  'goal': 287000.0,
  'launched': '2012-08-13 23:14:02',
  'pledged': 1465.0,
  'state': 'failed',
  'backers': 11.0,
  'country': 'US',
  'usd pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': ObjectId('61dd8dd43bd217711a71f536'),
  'ID': 993194166,
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2012-06-01',
  'goal': 250000.0,
  'launched': '2012-05-01 20:49:58',
  'pledged': 23680.54,
  'state': 'failed',
  'backers': 409.0,
  'country': 'US',
  'usd pledged': 23680.54,
  'usd_pledged_real': 23680.54},
 {'_id': ObjectId('61dd8dd43bd217711a71f53e'),
  'ID': 1147175344,
  'name': 'Shine On New World',
  'category': 'Theater',
  'main_category': 'Theater',
  'currency': 

## Question 6 
Compter le nombre de projet ayant "Sport" dans leur nom

In [196]:
collection.create_index([("name",  "text")])
collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [198]:
collection.count({"$text":{"$search":"Sport"}})

/tmp/ipykernel_15/2766307636.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  collection.count({"$text":{"$search":"Sport"}})


318